# Load all imports

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import task3 as helper
import net

# Globals, CNNs, and Device

In [2]:
oriPATH = 'C:/Users/Ryan/Desktop/machine-learning/part2/cnn'
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')
learningRates = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1]
numEpochs = 50

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

regNet = net.ReluNet()
earlyNet = net.ReluNet()
regNet = regNet.to(device)
earlyNet = earlyNet.to(device)

cuda:0


# MNIST training and validation set augmentation

In [3]:
valid_ratio = 0.3

noTransform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

transform = transforms.Compose([
    # define your data augmentation here!
    transforms.RandomRotation(degrees=60),
    transforms.RandomRotation(degrees=300),
    transforms.RandomRotation(degrees=30),
    transforms.RandomRotation(degrees=330),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

nb_train = int((1.0 - valid_ratio) * len(train_valid_dataset))
nb_valid =  int(valid_ratio * len(train_valid_dataset))

notTransformed = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=noTransform)
transformed = = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)

train_dataset, valid_dataset = torch.utils.data.dataset.random_split(notTransformed, [nb_train, nb_valid])
nt_trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=1000, shuffle=True, num_workers=2, pin_memory=True)
nt_validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=1000, shuffle=True, num_workers=2, pin_memory=True)

t_train_dataset, t_valid_dataset = torch.utils.data.dataset.random_split(transformed, [nb_train, nb_valid])
t_trainloader = torch.utils.data.DataLoader(t_train_dataset, batch_size=1000, shuffle=True, num_workers=2, pin_memory=True)
t_validloader = torch.utils.data.DataLoader(t_valid_dataset, batch_size=1000, shuffle=True, num_workers=2, pin_memory=True)

# Define the loss function and the optimizer.

In [4]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()

regOpt = optim.Adam(regNet.parameters(), lr=0.001)
earlyOpt = optim.Adam(earlyNet.parameters(), lr=0.001)

# Train the CNN and store the best model based on the validation loss.

In [5]:
import time
import os as OO
OO.mkdir(oriPATH + '/task3')
PATH = oriPATH + '/task3'

regTrainingLoss = []
regValidationLoss = []
earlyTrainingLoss = []
earlyValidationLoss = []

train, val = helper.runCNN(trainloader, device, regOpt, regNet, criterion, validloader, PATH, 'regNet', numEpochs)
earlyTrainingLoss.append(train)
earlyValidationLoss.append(val)
train, val = helper.runCNN_earlyStop(trainloader, device, earlyOpt, earlyNet, criterion, validloader, PATH, 'earlyNet', numEpochs)
regTrainingLoss.append(train)
regValidationLoss.append(val)

np.save(OO.path.join(PATH, 'regTrainingLoss.npy'), regTrainingLoss)
np.save(OO.path.join(PATH, 'regValidationLoss.npy'), regValidationLoss)
np.save(OO.path.join(PATH, 'earlyTrainingLoss.npy'), earlyTrainingLoss)
np.save(OO.path.join(PATH, 'earlyValidationLoss.npy'), earlyValidationLoss)

# Define the test dataset.

In [6]:
transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,))
])
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Infer on the whole test dataset.

In [9]:
testloader = torch.utils.data.DataLoader(testset, batch_size=1000, shuffle=False, num_workers=2, pin_memory=True)

import os as OO
PATH = oriPATH + '/task3'

accuracy = []

testNet = net.ReluNet()
testNet = testNet.to(device)
testNet.load_state_dict(torch.load(OO.path.join(PATH, 'regNet.pth')))
correct, total = helper.testCNN(testloader, testNet, device)
accuracy.append(100 * correct / total)
testNet2 = net.ReluNet()
testNet2 = testNet2.to(device)
testNet2.load_state_dict(torch.load(OO.path.join(PATH, 'earlyNet.pth')))
correct, total = helper.testCNN(testloader, testNet, device)
accuracy.append(100 * correct / total)

sTrain = np.load(OO.path.join(PATH, 'regTrainingLoss.npy'))
rTrain = np.load(OO.path.join(PATH, 'earlyTrainingLoss.npy'))

regNumEpochs = len(sTrain[0])
earlyNumEpochs = len(rTrain[0])

print("regular network: {} epochs %.3F %% accuracy".format(regNumEpochs) % accuracy[0])
print("early stop network: {} epochs %.3F %% accuracy".format(earlyNumEpochs) % accuracy[1])


10 epochs for regular network: 98.710 % accuracy
10 epochs for early stop network: 98.710 % accuracy


# Graph Training Losses and Accuracy

In [10]:
import numpy as np
import matplotlib.pyplot as plt

epochs = range(0, 10)
fig, ax = plt.subplots(1, 2)
fig.set_size_inches(20, 10)

ax[0].plot(epochs, sTrain[0], label='sigmoid')
ax[0].plot(epochs, rTrain[0], label='relu')

ax[1].plot(epochs, sigAccuracy, label='sigmoid')
ax[1].plot(epochs, relAccuracy, label='relu')

ax[0].legend(loc="lower left")
ax[1].legend(loc="upper left")

ax[0].set_title("Training Loss per Epoch")
ax[1].set_title("Accuracy per Epoch")

ax[0].set_xlabel("Epoch")
ax[0].set_ylabel("Training Loss")
ax[1].set_xlabel("Epoch")
ax[1].set_ylabel("Training Loss")